# Structures of Networks - workgroup session 6

The structure of this notebook follows that of the associated problem sheet.
Code blocks (cells) are already available for (sub)problems that involve coding, but you can always add more.
Some of these code blocks contain coding-related tips, in the form of comments.

## 1.a.

In [ ]:
import networkx as ntx
import random as rn
import numpy as np
import math as mt
import matplotlib.pyplot as plt

rn.seed(1)
np.random.seed(1)

# randomly generates a list of feature vectors (LFV), containing n vectors of F entries each, where each entry is a binary random (unbiased) variable
def random_LFV(n, F):
    x = []    
    for i in range(0, n):
        xi = []
        for f in range(0, F):
            xi.append(rn.randint(0, 1))
        x.append(xi)
        
    return x

# randomly generates a list of feature vectors with categories (LFVC), containing n vectors of F entries (feature/variable values)
# the number of values that each feature may take matches the number of categories, specified by argument C
# each category has un underlying, latent feature vector, populated by one value, constant across features and distinct from the (constant) values associated to other latent vectors
# each vector is randomly assigned to one category and generated as a partial copy of that category's latent vector
# for each vector, the values of D randomly selected features are copied from its category's latent vector, while the other are generated randomly
# returns the list of generated feature vectors, along with a list of associated integers encoding the ground-truth partition - each entry identifies the a-priori category of the vector
def random_LFVC(n, F, C, D):
    # list of feature vectors
    x = []
    # ground truth partition, containing one integer for each feature vector
    gtp = [] 

    for i in range(0,n):
        xi = []
        # picking category
        c = rn.choice(range(0,C))
        gtp.append(c)
        # sampling without replacement, using rn.sample(), D features that are to be copied from the category's latent vector
        lf = sorted(rn.sample(range(0,F), D))
        for a in range(0,F):
            genrand = False
            if(len(lf) > 0):
                if(a == lf[0]):
                    xi.append(c)
                    del lf[0]
                else:
                    genrand = True
            else:
                genrand = True
            if(genrand):
                xi.append(rn.choice(range(0,C)))
        x.append(xi)
    
    return x, gtp

# applies a joint sorting operation on data compliant with the output of random_LFVC function above
# carries out sorting for increasing values of the ground truth partition (gtp) list
# returns the consistently sorted counterparts of the set of feature vectors (x) and ground truth partition (gtp), along with the list of vector ids (consistent with the initial ordering)
def joint_sort(x, gtp):

    ids = range(0,n)

    # assembling list of tuples
    lt = []
    for i in range(0,n):
        lt.append((x[i], gtp[i], ids[i]))

    # sorting the list of tuples according to gtp value
    lt_s = sorted(lt, key=lambda t: t[1])  

    # unpacking the sorted list of tuples
    x_s = []
    gtp_s = []
    ids_s = []
    for i in range(0,n):
        (x_i, gtp_i, ids_i) = lt_s[i]
        x_s.append(x_i)
        gtp_s.append(gtp_i)
        ids_s.append(ids_i)
    
    return x_s, gtp_s, ids_s

# computes the (Hamming) similarity between two (binary) vectors of equal lenghts
def similarity(x1, x2):
    F = len(x1)
    nmatch = 0.0
    for a in range(0, F):
        nmatch += int(x1[a] == x2[a])
    return float(nmatch)/F

# computes the lmbd-adjusted (Hamming similarity between two (binary) vectors of equal lengths
def adjusted_similarity(x1, x2, lmbd1, lmbd2):
    return (similarity(x1, x2))**(lmbd1*lmbd2)
    

# generates a random graph from a list of feature vectors x, using a lmbd-power-adjusted Hamming similarity, which is interpreted as a link probability
# here, lmdb (lambda) is a list of positive-real values, one for each of the feature vectors in x (and thus for each of the nodes in the graph being generated) 
def LFV_graph(x, lmbd):
    n = len(x)
    G = ntx.empty_graph(n)
    for i in range(0, n-1):
        for j in range(i+1, n):
            # probability of generating link (i,j):
            p = adjusted_similarity(x[i], x[j], lmbd[i], lmbd[j])
            r = rn.random()
            if(r < p):
                ntx.add_path(G, [i,j])
    return G

# computes the the mean of the lmbd-adjusted (Hamming) similarities between all pairs of feature vectors in x
# this is equivalent to the expected network density, given x and lmbd
def mean_adjusted_similarity(x, lmbd):
    sum = 0.0
    n = len(x)
    for i in range(0, n-1):
        for j in range(i+1, n):
            sum += adjusted_similarity(x[i], x[j], lmbd[i], lmbd[j])
    return sum*2.0/n/(n-1)

# computes the expectation value of the degree of node i, given x and lmbd
def expected_degree(x, lmbd, i):
    sum = 0.0
    for j in range(0, len(x)):
        if(j != i):
            sum += adjusted_similarity(x[i], x[j], lmbd[i], lmbd[j])
    return sum

# computes area under the line segment between points c1, c2, where each point is encoded as a 2D coordinate pair (tuple)
def slice_area(coord1, coord2):
    (x1, y1) = coord1
    (x2, y2) = coord2
    return (y1+y2)*(x2-x1)/2

# computes area under a sequence of line segments defined by coords, which is a list of coordinate pairs
# the l'th segment goes between coordinate pairs c[l] and c[l+1]
def combined_area(coords):
    A = 0.0
    for l in range(0,len(coords)-1):
        A += slice_area(coords[l], coords[l+1])
    return A

# adjusts the y-components of all coordinate pairs in coords so that the latter can be interprted as a (properly normalized) probability density function with a piecewise-linear shape
# the coordinates have to be sorted so that the x-components are strictly increasing
def normalize(coords):
    A = combined_area(coords)
    for l in range(0, len(coords)):
        (x,y) = coords[l]
        coords[l] = (float(x), float(y)/A)
    return coords

# generates a random number sampled from a distribution with a density function that takes a piecewise-linear shape specified by coordinate (x,y) pairs in coords list
# each y-entry corresponds to the probability density for the respective x-entry
# the l'th segment of the density function goes between coordinate pairs c[l] and c[l+1]
# the coordinates have to be sorted so that the x-components are strictly increasing
# the y-coordinates are automatically rescaled, by the same factor, in order to ensure that the distribution is normalized  
def rand_piecewise(coords):

    coords = normalize(coords)
    
    r = rn.random()
    rr = -1.0
    
    searching = True
    l = 0
    
    while(searching):
        A = slice_area(coords[l], coords[l+1])

        if(r <= A):
            searching = False
            (x1, y1) = coords[l]
            (x2, y2) = coords[l+1]
            a = (y2-y1)/(x2-x1)
            b = (x2*y1 - x1*y2)/(x2-x1)
            if(a == 0.0):
                rr = x1 + r/y1
            else:
                aa = a
                bb = (b - a*x1 + y1)
                cc = - (x1*(b+y1) + 2*r)
                rr = (- bb + (bb**2 - 4*aa*cc)**(0.5))/(2.0*aa)
        else:
            r -= A

        l += 1
        
        if(l == len(coords)-1):
            searching = False

    return rr


## 1.b

In [ ]:
# specify parameters

# invoke generation function

# invoke sorting function

# if ids_s is the sorted list of feature vector ids, then the following code gives you the permutation used during the sorting:
# perm = [0]*n
# for i in range(0,n):
#    perm[ids_s[i]] = i

In [ ]:
# compute the (sorted) similarity matrix s_s

In [ ]:
# visualize the (sorted) similarity matrix s_s using:
# plt.matshow(s_s)
# plt.colorbar()
# plt.show()


## 1.c.

In [ ]:
# specify the lambda distribution via a set of coordinates pairs like:
# lmbd_pdf_coords = [(0.5, 0.0), (1.0, 1.0), (1.5, 0.0)]

# then generate a vector of lambda values using something like:
# lmbd = []
# for i in range(0,n):
#     lmbd.append(rand_piecewise(lmbd_pdf_coords))



In [ ]:
# visualize the sampled lambda values using a histogram

In [ ]:
# compute the (sorted) adjusted similarity matrix sa_s like in 1.b

In [ ]:
# visualize the (sorted) adusted similarity matrix sa_s like in 1.b

## 1.d.

In [ ]:
# generate a random network FDRG

In [ ]:
# initialize a (sorted) adjacency matrix filled it with zeroes:
# a_s = np.empty((n,n))
# for i in range(0,n):
#     for j in range(0,n):
#         a_s[i][j] = 0.0
# then set to 1.0 the entries for which there is a link in FDRG, while accounting for the sorting carried out in 1.b.

In [ ]:
# visualize the (sorted) adjacency matrix a_s like in 1.b

## 1.e.

In [ ]:
# run the Louvain method on the FDRG network generated above

In [ ]:
# convert the output to the format inherent to the ground-truth partition generated in 1.b

## 1.f.

In [ ]:
# generate the joint distribution matrix

In [ ]:
# visualize the joint distribution matrix like in 1.b

## 1.g